In [1]:
# -*- coding:utf-8 -*- 

In [2]:
import pandas as pd
import traitlets
from IPython.display import display,update_display
import ipywidgets as widgets
import bqplot as bp
import random

In [3]:
import sys 
sys.path.append('libs/')
from table_v3 import TableChart

In [4]:
class EtlCommand(traitlets.HasTraits):
    
    _df = pd.DataFrame()
    #_data = traitlets.Dict()
    #_header = traitlets.Dict()
    _table = None
    _show = False
    _GUI = None
 
    def __init__(self,df):
        self._df = df
        self._data = df.values.tolist()
        self._header = df.columns.values.tolist()
        self._table = TableChart(self._header,self._data)
    
    def drop(self,**kwargs):
        col_value = kwargs.get('col')
        if col_value and col_value in self._header:
            self._df.drop(col_value,axis=1,inplace=True)
            self.update()
        else:
            raise Exception('col is not valid, must be in list[{}]'.format(self._df.columns.values.tolist()))
    
    def update(self):
        #self._data = df.values.tolist()
        #self._header = df.columns.values.tolist()
        if self._show:
            self._table.model.data = self._df.values.tolist()
            self._table.model.header = self._df.columns.values.tolist()
            kwargs={'id':random.random(),'operation':'drop','col':df.columns.values.tolist()}
            #update_display(self._updatehandle,display_id='commandGUI')
            self._GUI.cmdargs = kwargs
    
    def showTable(self,enable=True):
        if isinstance(enable,bool):
            self._show = enable
        else:
            raise Exception('display must be bool value')
        if self._show:
            self.show_handle = display(self._table)
        else:
            self._table.close()
            self._show = False
     
    def showGUI(self):
        self._GUI = EtlGUI(self,{'operation':'drop','col':df.columns.values.tolist()})
        self._updatehandle = display(self._GUI,display_id='commandGUI')


In [44]:
class EtlGUI(widgets.Box):
    
    _cmds = None
    _args = None
    _argsdict = traitlets.Dict()
    
    def __init__(self,cmds,kwargs=None):
        super(EtlGUI,self).__init__()
        self._cmds = cmds
        self._argsdict = kwargs 
        self._args = self.initArgs()
        self._btn_ok = widgets.Button(description=u'确认',layout=widgets.Layout(width='10%',))
        self.children = [self._lb_operation,self._args,self._btn_ok]
        self.layout = widgets.Layout(width="100%",display='flex-flow',flex_flow='row', justify_content=u'flex-start')
        self._btn_ok.on_click(self.execute)
        
    def execute(self,a):
        cmd_label = self._lb_operation.value
        action_handler = getattr(self._cmds,cmd_label)
        if not action_handler:
            raise Exception('label:{} is not a valid ETL command '.format(cmd_label))
        args = {}
        for arg in self._args.children:
            arg_name = arg.children[0].value
            arg_value = arg.children[1].value
            args[arg_name] = arg_value
        action_handler(**args)
    
    @property
    def cmdargs(self):
        return self._argsdict
    
    @cmdargs.setter
    def cmdargs(self,kwargs=None):
        self._argsdict = kwargs
    
    
    
    @traitlets.observe('_argsdict')
    def updateArgs(self,change=None):
        
        if not self._args:
            return 
        for arg in self._args.children:
            arg_name = arg.children[0].value
            if arg_name in self._argsdict:
                arg.children[1].options =  self._argsdict[arg_name]
                arg.children[1].value =  None

        
    def initArgs(self):
        kwargs = self._argsdict
        print(kwargs)
        if 'id' in kwargs:
            del kwargs['id']
        width_auto =  '{}%'.format(100 / len(kwargs))
        input_args = []

        for arg in kwargs:
            value = kwargs[arg]
            if arg ==u'operation':
                self._lb_operation = widgets.Label(value,layout=widgets.Layout(width='10%'))
                continue
            if isinstance(value,int):
                widget_arg = widgets.IntText(value)
            elif isinstance(value,list):
                widget_arg = widgets.Dropdown(options=value,value=None)
                if arg =='col':
                    widget_arg.observe(self.colChoose,'value')
                    
            elif isinstance(value,str):
                widget_arg = widgets.Text(value)
            group_arg = widgets.HBox(layout=widgets.Layout(width=width_auto))
            width_arg_label = '{}px'.format(len(arg)*16)
            group_arg.children = [widgets.Label(arg,layout=widgets.Layout(width=width_arg_label)),widget_arg]
            input_args.append(group_arg)
        
        arg_hbox = widgets.HBox(layout=widgets.Layout(width='80%'))
        arg_hbox.children = input_args
        return arg_hbox
    
    def colChoose(self,change):
        value=change['new']
        self._cmds._table.colChoose([value])
        

class ColWidget(widgets.Box):
    
    value = traitlets.Unicode()
    def __init__(self,label='col',options=None,placehold='please choose a column'):
        super(ColWidget,self).__init__()
        
        self.layout = widgets.Layout(width="100%",display='flex-flow',flex_flow='row', justify_content=u'flex-start')
        self.placehold = placehold
        labels = widgets.Label(label,layout =widgets.Layout(width='64px') )
        if not(isinstance(options,list)):
            raise Exception('options must be a list')
        self.inputs = widgets.Dropdown(options =[self.placehold]+options,value=self.placehold )
        self.children = [labels,self.inputs]
        self.inputs.observe(self.eventColChoose,'value')   
        
    def eventColChoose(self,change):
        print('eventColChoose:' + change['new'])
        if change['new'] != self.placehold:
            self.value = change['new']

class FilterWidget(widgets.Box):
    value = traitlets.Unicode()
    apply = traitlets.Bool(False)
    
    def __init__(self,options=None,dtypes=None,placehold='please choose a column'):
        super(FilterWidget,self).__init__()
        self._optType = dict(zip(options,dtypes))
        self.layout = layout=widgets.Layout(display='flex-flow',flex_flow='column')
        cmdSection = widgets.Box(layout=widgets.Layout(display='flex-flow',flex_flow='row', justify_content=u'flex-start'))
        self.wColWidget = ColWidget(options=options,placehold=u'',label='请选择列')
        self.btnAdd = widgets.Button(description=u'增加条件')
        self.btnApply = widgets.Button(description=u'应用筛选')
        self.btnCancel = widgets.Button(description=u'取消筛选')
        cmdSection.children=[self.wColWidget,self.btnAdd,self.btnApply,self.btnCancel]
        self.children = [cmdSection]
        self.btnAdd.on_click(self.andFilter,False)
        self.btnApply.on_click(self.filterApply,False)
        self.btnCancel.on_click(self.filterCancel,False)
    
    def andFilter(self,a):
        partCol = widgets.Label(self.wColWidget.value,layout=widgets.Layout(width='100px'))
        partOp = widgets.Dropdown(options=['==','>','>=','<','<=','<>','in','between'],layout=widgets.Layout(width='100px'))
        partInput = widgets.Text(placehold=u'请输入筛选范围')
        partAnd = widgets.Label('and',layout=widgets.Layout(width='100px'))
        addCondition = widgets.Box(layout=widgets.Layout(display='flex-flow',flex_flow='row', justify_content=u'flex-start'))
        addCondition.children = [partCol,partOp,partInput,partAnd ]
        self.children = [c for c in self.children] + [addCondition]
    
    def filterApply(self,a):
        cond = []
        for b in f.children[1:]:
            field = b.children[0].value
            op = b.children[1].value
            value = b.children[2].value
            fldType = self._optType.get(field)
            if (str(fldType) in ['category','object']):
                cond.append("({} {} '{}')".format(field , op ,value)) 
            #elif Todo:handle other type data    
            else:
                cond.append("({} {} {})".format(field , op ,value))
        self.value = ' and '.join(cond)
        self.apply = True
        
    def filterCancel(self,a):
        self.value = ''
        self.apply = False    
        

    

In [7]:
class DropWidget(EtlGUI):
    _operation = 'drop'
    
    def __init__(self,cmds,kwargs=None):
        super(DropWidget,self).__init__(self,cmds,kwargs=None)    

In [16]:
df = pd.read_csv('../data/iris.csv')

In [17]:
cmd = EtlCommand(df)
cmd.showTable()

A Jupyter Widget

In [18]:
cmd.showGUI()

{'operation': 'drop', 'col': ['petalLength', 'petalWidth', 'sepalLength', 'sepalWidth', 'species']}


A Jupyter Widget

In [41]:

f =FilterWidget(options=df.columns.values.tolist(),dtypes=['float64','float64','float64','float64','category'],placehold=u'请选择')
f

A Jupyter Widget

eventColChoose:petalWidth
eventColChoose:


In [42]:
pd.Categorical

pandas.core.categorical.Categorical